In [1]:
import numpy
!pip install spotipy
import spotipy
# from google.colab import drive
# drive.mount('/content/drive')


In [76]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.002, beta=0.1):
  '''
  R: rating matrix
  P: |U| * K (User features matrix)
  Q: |D| * K (Item features matrix)
  K: latent features
  steps: iterations
  alpha: learning rate
  beta: regularization parameter'''
  Q = Q.T
  for step in range(steps):
      for i in range(len(R)):
          for j in range(len(R[i])):
              if R[i][j] > 0:
                  # calculate error
                  eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                  for k in range(K):
                      # calculate gradient with a and beta parameter
                      P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                      Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
      eR = numpy.dot(P,Q)
      e = 0
      for i in range(len(R)):
          for j in range(len(R[i])):
              if R[i][j] > 0:
                  e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                  for k in range(K):
                      e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
      # 0.001: local minimum
      if e < 0.001:
          break
  return P, Q.T



def find_unknown(data):
  all_unknown_songs = []
  for i, user in enumerate(data):
    # print("i: ", i, "user:", user)
    all_unknown_songs.append([])
    for j, rating in enumerate(user):
      # if rating==0: all_unknown_songs[i].append()
      if rating==0:
        # print("Song ", j)
        all_unknown_songs[i].append(j)
  return all_unknown_songs


def find_unknown_user(data, user):
  unknown_songs =[]
  user_data = data[user]
  for i, rating in enumerate(user_data):
    # if rating==0: all_unknown_songs[i].append()
    if rating==0:
      unknown_songs.append(i)
  return unknown_songs

def generate_scores(data):
  data = numpy.array(data)
  #N: num of User
  N = len(data)
  # M: num of Movie
  M = len(data[0])
  # Num of Features
  K = 3

  P = numpy.random.rand(N,K)
  Q = numpy.random.rand(M,K)

  nP, nQ = matrix_factorization(data, P, Q, K)

  nR = numpy.dot(nP, nQ.T)
  return nR


def recommendations(ratings, scores, id):
  #ratings - user entered data
  #unknown_indices - indices of users unknown
  #scores - generated predictions
  #five_recs - (index, score)
  unknown_indices = find_unknown(ratings)[id]
  user_scores = scores[id]
  five_recs = [(0,0),(0,0),(0,0),(0,0),(0,0)]
  for i in unknown_indices:
    # print("i:", i)
    i_score = user_scores[i-1]
    min_tuple = min(five_recs, key = lambda t: t[1])
    if i_score > min_tuple[1]:
      five_recs.remove(min_tuple)
      five_recs.append((song_index[i],i_score))
      five_recs.sort(key=lambda x: x[1])
      five_recs.reverse()
  return five_recs
      

def rm_duplicates2(matrix):
  rt_matrix = matrix.copy()
  for entry in rt_matrix:
    temp = rt_matrix.copy()
    temp.remove(entry)
    for o_entry in temp:
      if(o_entry == entry): 
        rt_matrix.remove(entry)
  return rt_matrix


def rm_duplicates(matrix):
  uniq = []
  for row in matrix:
    if(row not in uniq):
      uniq.append(row)
  return uniq
  

def find_losers(matrix): 
  losers = []
  num_users = len(matrix)
  for song in range(len(matrix[0])):
    # print("song index: ", song)
    zero_count=0
    for user in matrix:
      if user[song] == 0:
        zero_count +=1
    if (zero_count/num_users) >= 0.75:
      losers.append(song)
  return losers


def remove_losers(matrix): 
  losers = find_losers(matrix)
  losers.reverse()
  # print(losers)
  rt_matrix = matrix.copy()
  for loser in losers:
    print("Loser: " ,loser)
    for user in matrix :
      print(user)
      del(user[loser])
  return rt_matrix


def Excel2Matrix(self, fis):
  wrkbk = xlrd.open_workbook(fis)
  sheet = wrkbk.sheet_by_index(0)
  matrix = []

  for row in range(sheet.nrows):
      _row = []
      for col in range(0, sheet.ncols):
          _row.append(sheet.cell_value(row,col))
      matrix.append(_row)
  return matrix

  
def rm_duplicates(m):
    rt_matrix = m.copy()
    for entry in rt_matrix:
        temp = rt_matrix.copy()
        temp.remove(entry)
        for o_entry in temp:
            if (o_entry == entry):
                rt_matrix.remove(entry)
    return rt_matrix



    


In [69]:
last_data = [[5.0, 5.0, 5.0, 4.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 5.0, 5.0, 1.0, 0.0, 1.0, 2.0, 1.0, 0.0, 1.0, 5.0, 0.0, 1.0, 1.0, 2.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 5.0, 4.0, 0.0, 1.0, 5.0, 0.0, 2.0, 4.0, 5.0, 1.0, 5.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 2.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [4.0, 4.0, 4.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 4.0, 4.0, 0.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 4.0, 3.0, 0.0, 5.0, 3.0, 5.0, 0.0, 4.0, 5.0, 0.0, 0.0], [5.0, 4.0, 1.0, 4.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 0.0, 5.0, 5.0, 4.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 3.0, 0.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 1.0], [4.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 5.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 4.0, 5.0, 5.0, 0.0, 0.0, 4.0, 5.0, 5.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 4.0, 4.0, 0.0, 4.0, 5.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0], [4.0, 4.0, 4.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 3.0, 5.0, 3.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 5.0, 0.0, 3.0, 0.0, 0.0, 0.0, 2.0, 0.0, 4.0, 0.0, 3.0, 2.0, 0.0, 2.0, 3.0, 3.0, 0.0, 0.0], [0.0, 1.0, 5.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 3.0, 0.0, 5.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 5.0, 0.0, 5.0, 4.0, 0.0, 0.0], [0.0, 2.0, 2.0, 2.0, 5.0, 0.0, 0.0, 4.0, 4.0, 0.0, 4.0, 4.0, 0.0, 5.0, 5.0, 4.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 5.0, 4.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 4.0, 3.0, 0.0, 4.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, 5.0, 3.0, 0.0, 2.0, 4.0, 0.0, 3.0, 5.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 4.0, 5.0, 5.0, 4.0, 0.0, 3.0, 4.0, 0.0, 0.0], [5.0, 4.0, 2.0, 3.0, 5.0, 0.0, 4.0, 3.0, 3.0, 0.0, 4.0, 0.0, 3.0, 4.0, 3.0, 3.0, 0.0, 3.0, 0.0, 4.0, 0.0, 3.0, 0.0, 5.0, 3.0, 3.0, 0.0, 0.0, 0.0, 3.0, 4.0, 0.0, 0.0, 0.0, 2.0, 4.0, 0.0, 3.0, 4.0, 3.0, 3.0, 3.0, 0.0, 3.0, 2.0, 0.0, 4.0, 4.0, 2.0, 5.0, 5.0, 4.0, 3.0, 3.0, 4.0, 4.0, 0.0, 3.0, 0.0, 3.0, 0.0, 4.0, 3.0, 3.0, 4.0, 5.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 0.0, 0.0, 0.0], [3.0, 4.0, 1.0, 3.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 3.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 2.0, 0.0, 3.0, 0.0, 5.0, 0.0, 4.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 2.0, 0.0, 2.0, 0.0, 0.0, 0.0], [5.0, 3.0, 1.0, 3.0, 2.0, 2.0, 3.0, 3.0, 1.0, 2.0, 4.0, 1.0, 4.0, 3.0, 2.0, 3.0, 4.0, 2.0, 2.0, 4.0, 2.0, 3.0, 2.0, 2.0, 1.0, 1.0, 3.0, 2.0, 1.0, 4.0, 5.0, 2.0, 2.0, 2.0, 3.0, 2.0, 1.0, 3.0, 3.0, 2.0, 4.0, 5.0, 5.0, 2.0, 5.0, 1.0, 3.0, 3.0, 1.0, 4.0, 4.0, 2.0, 4.0, 3.0, 4.0, 5.0, 1.0, 4.0, 3.0, 3.0, 2.0, 2.0, 2.0, 1.0, 4.0, 2.0, 2.0, 5.0, 1.0, 4.0, 3.0, 1.0, 1.0, 2.0, 3.0], [0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 4.0, 5.0, 3.0, 0.0, 2.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 3.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0], [3.0, 4.0, 0.0, 0.0, 4.0, 0.0, 5.0, 4.0, 5.0, 5.0, 4.0, 5.0, 0.0, 4.0, 4.0, 4.0, 0.0, 4.0, 3.0, 4.0, 0.0, 0.0, 4.0, 4.0, 0.0, 4.0, 0.0, 5.0, 0.0, 3.0, 5.0, 0.0, 5.0, 4.0, 0.0, 4.0, 4.0, 2.0, 3.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 4.0, 4.0, 5.0, 3.0, 0.0, 3.0, 4.0, 5.0, 3.0, 5.0, 5.0, 3.0, 0.0, 5.0, 3.0, 2.0, 5.0, 0.0, 5.0, 3.0, 3.0, 0.0, 5.0, 0.0, 0.0, 5.0, 5.0], [0.0, 4.0, 5.0, 0.0, 0.0, 4.0, 5.0, 5.0, 5.0, 5.0, 4.0, 0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 5.0, 0.0, 5.0, 4.0, 4.0, 5.0, 5.0, 0.0, 3.0, 5.0, 0.0, 0.0, 0.0, 1.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 0.0, 4.0, 4.0, 5.0, 5.0, 4.0, 4.0, 3.0, 4.0, 3.0, 0.0, 5.0, 5.0, 2.0, 0.0, 4.0, 4.0, 0.0, 5.0, 5.0, 0.0, 4.0, 0.0, 5.0, 0.0, 4.0, 4.0, 0.0, 3.0], [0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 4.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 4.0, 4.0, 3.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 4.0, 4.0, 4.0, 0.0, 2.0, 4.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 3.0, 4.0, 0.0, 4.0, 4.0, 0.0, 0.0], [3.0, 3.0, 2.0, 2.0, 0.0, 0.0, 5.0, 3.0, 4.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 5.0, 0.0, 3.0, 0.0, 0.0, 5.0, 4.0, 5.0, 3.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 3.0, 5.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 5.0, 5.0, 2.0, 4.0, 3.0, 3.0, 0.0, 5.0, 0.0, 0.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 0.0, 0.0], [3.0, 4.0, 2.0, 2.0, 5.0, 0.0, 5.0, 0.0, 4.0, 0.0, 4.0, 0.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, 0.0, 5.0, 0.0, 5.0, 4.0, 0.0, 4.0, 0.0, 0.0, 3.0, 4.0, 0.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 4.0, 2.0, 0.0, 0.0, 1.0, 3.0, 0.0, 0.0, 0.0, 5.0, 5.0, 5.0, 3.0, 0.0, 3.0, 4.0, 5.0, 4.0, 5.0, 4.0, 5.0, 5.0, 5.0, 0.0, 4.0, 5.0, 4.0, 0.0, 3.0, 5.0, 4.0, 3.0, 4.0, 4.0, 0.0, 0.0], [3.0, 3.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 5.0, 0.0, 4.0, 3.0, 5.0, 5.0, 5.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 3.0, 0.0, 4.0, 0.0, 2.0, 5.0, 0.0, 0.0, 4.0, 2.0, 5.0, 2.0, 0.0, 5.0, 5.0, 4.0, 5.0, 0.0, 1.0, 0.0, 5.0, 0.0, 4.0, 0.0, 5.0, 0.0, 0.0, 0.0, 5.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 5.0, 5.0, 4.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 3.0, 0.0, 5.0, 0.0, 0.0, 4.0, 4.0, 0.0, 4.0, 5.0, 0.0, 4.0, 5.0, 0.0, 4.0, 0.0, 5.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0], [0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 4.0, 0.0, 4.0, 4.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 3.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 3.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 4.0, 4.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 5.0, 0.0, 5.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 4.0, 5.0, 0.0, 5.0], [3.0, 4.0, 4.0, 3.0, 5.0, 5.0, 4.0, 4.0, 4.0, 0.0, 3.0, 2.0, 3.0, 5.0, 4.0, 4.0, 0.0, 3.0, 0.0, 3.0, 0.0, 4.0, 0.0, 5.0, 0.0, 3.0, 3.0, 0.0, 3.0, 5.0, 4.0, 0.0, 0.0, 0.0, 2.0, 4.0, 0.0, 3.0, 3.0, 0.0, 4.0, 3.0, 0.0, 0.0, 3.0, 0.0, 4.0, 0.0, 2.0, 5.0, 5.0, 4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 5.0, 0.0, 3.0, 0.0, 0.0, 4.0, 3.0, 4.0, 4.0, 0.0, 5.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.0, 2.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 5.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 2.0, 0.0, 2.0, 3.0, 2.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 4.0, 0.0, 4.0, 4.0, 3.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0], [0.0, 3.0, 4.0, 1.0, 5.0, 0.0, 3.0, 2.0, 0.0, 3.0, 5.0, 0.0, 3.0, 5.0, 4.0, 5.0, 1.0, 0.0, 2.0, 1.0, 0.0, 4.0, 3.0, 4.0, 4.0, 5.0, 0.0, 2.0, 0.0, 4.0, 4.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 4.0, 4.0, 2.0, 0.0, 4.0, 5.0, 5.0, 4.0, 3.0, 4.0, 2.0, 5.0, 2.0, 4.0, 4.0, 3.0, 4.0, 5.0, 4.0, 4.0, 1.0, 4.0, 1.0, 1.0, 5.0, 4.0, 0.0, 3.0, 0.0, 3.0, 0.0, 4.0, 3.0, 4.0, 2.0, 0.0, 2.0], [5.0, 5.0, 5.0, 3.0, 5.0, 0.0, 4.0, 0.0, 5.0, 5.0, 4.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 5.0, 0.0, 5.0, 0.0, 5.0, 4.0, 3.0, 0.0, 5.0, 5.0, 5.0, 5.0, 0.0, 0.0, 5.0, 4.0, 0.0, 4.0, 4.0, 5.0, 0.0, 2.0, 4.0, 5.0, 5.0, 0.0, 0.0, 5.0, 4.0, 5.0, 3.0, 4.0, 5.0, 3.0, 4.0, 5.0, 4.0, 0.0, 5.0, 3.0, 0.0, 4.0, 4.0, 5.0, 4.0, 2.0, 5.0, 4.0, 4.0, 3.0, 4.0, 5.0, 3.0, 0.0], [0.0, 5.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 3.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 3.0, 5.0, 5.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0], [0.0, 3.0, 4.0, 2.0, 0.0, 0.0, 2.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 4.0, 2.0, 0.0, 5.0, 4.0, 0.0, 5.0, 5.0, 2.0, 5.0, 1.0, 3.0, 0.0, 0.0], [5.0, 5.0, 1.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 3.0, 1.0, 0.0, 3.0, 2.0, 0.0, 3.0, 5.0, 0.0, 2.0, 0.0, 2.0, 3.0, 5.0, 0.0, 3.0, 4.0, 0.0, 1.0, 5.0, 2.0, 0.0, 3.0, 0.0, 5.0, 3.0, 5.0, 4.0, 4.0, 0.0, 0.0, 1.0, 0.0, 5.0, 2.0, 0.0, 1.0, 4.0, 3.0, 0.0, 3.0, 0.0, 0.0, 0.0], [3.0, 1.0, 1.0, 1.0, 5.0, 4.0, 5.0, 3.0, 3.0, 3.0, 4.0, 5.0, 0.0, 4.0, 4.0, 4.0, 1.0, 4.0, 0.0, 4.0, 3.0, 0.0, 2.0, 5.0, 5.0, 3.0, 4.0, 4.0, 0.0, 5.0, 4.0, 3.0, 0.0, 4.0, 1.0, 3.0, 0.0, 0.0, 0.0, 3.0, 2.0, 0.0, 4.0, 0.0, 2.0, 1.0, 5.0, 2.0, 3.0, 5.0, 5.0, 4.0, 3.0, 1.0, 5.0, 4.0, 3.0, 4.0, 4.0, 5.0, 0.0, 4.0, 3.0, 5.0, 5.0, 4.0, 0.0, 4.0, 4.0, 5.0, 4.0, 3.0, 3.0, 0.0, 0.0], [4.0, 3.0, 1.0, 3.0, 5.0, 1.0, 2.0, 4.0, 1.0, 3.0, 5.0, 5.0, 1.0, 4.0, 4.0, 4.0, 1.0, 5.0, 1.0, 5.0, 2.0, 1.0, 1.0, 5.0, 5.0, 3.0, 3.0, 4.0, 1.0, 5.0, 5.0, 2.0, 1.0, 1.0, 1.0, 4.0, 0.0, 2.0, 3.0, 2.0, 4.0, 1.0, 2.0, 1.0, 2.0, 4.0, 5.0, 2.0, 3.0, 5.0, 5.0, 5.0, 4.0, 4.0, 4.0, 5.0, 2.0, 5.0, 5.0, 3.0, 2.0, 2.0, 4.0, 1.0, 4.0, 5.0, 4.0, 5.0, 4.0, 5.0, 1.0, 4.0, 3.0, 1.0, 3.0], [1.0, 3.0, 2.0, 1.0, 5.0, 0.0, 4.0, 4.0, 3.0, 0.0, 5.0, 5.0, 4.0, 3.0, 5.0, 4.0, 0.0, 4.0, 4.0, 3.0, 0.0, 1.0, 0.0, 5.0, 5.0, 5.0, 4.0, 3.0, 0.0, 4.0, 5.0, 2.0, 0.0, 0.0, 1.0, 5.0, 0.0, 3.0, 2.0, 4.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 3.0, 2.0, 3.0, 5.0, 5.0, 5.0, 0.0, 2.0, 5.0, 4.0, 4.0, 5.0, 5.0, 1.0, 0.0, 5.0, 0.0, 0.0, 5.0, 3.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 5.0, 5.0, 0.0, 0.0, 0.0, 4.0, 3.0, 4.0, 0.0, 5.0, 5.0, 0.0, 3.0, 0.0, 0.0, 5.0, 5.0, 5.0, 5.0, 3.0, 4.0, 0.0, 0.0, 2.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 4.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 3.0, 0.0, 0.0, 3.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 3.0, 4.0, 0.0, 0.0, 2.0, 4.0, 1.0, 4.0, 4.0, 0.0, 0.0], [4.0, 4.0, 3.0, 4.0, 4.0, 0.0, 0.0, 0.0, 2.0, 0.0, 3.0, 3.0, 5.0, 3.0, 5.0, 4.0, 5.0, 4.0, 3.0, 4.0, 4.0, 5.0, 3.0, 4.0, 0.0, 3.0, 0.0, 3.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 4.0, 3.0, 0.0, 4.0, 3.0, 4.0, 2.0, 5.0, 5.0, 4.0, 5.0, 3.0, 0.0, 3.0, 4.0, 4.0, 5.0, 4.0, 0.0, 4.0, 0.0, 5.0, 5.0, 2.0, 0.0, 5.0, 0.0, 4.0, 4.0, 4.0, 4.0, 3.0, 0.0, 0.0, 3.0, 0.0, 4.0, 3.0, 5.0, 2.0, 5.0], [3.0, 5.0, 5.0, 4.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 5.0, 4.0, 5.0, 4.0, 3.0, 0.0, 0.0, 3.0, 4.0, 0.0, 3.0, 5.0, 0.0, 0.0, 4.0, 0.0, 5.0, 4.0, 0.0, 5.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 4.0, 5.0, 4.0, 0.0, 4.0, 0.0, 5.0, 0.0, 5.0, 0.0, 3.0, 4.0, 0.0, 0.0], [5.0, 5.0, 0.0, 5.0, 0.0, 2.0, 5.0, 0.0, 2.0, 3.0, 5.0, 0.0, 3.0, 3.0, 4.0, 0.0, 5.0, 3.0, 0.0, 5.0, 5.0, 5.0, 0.0, 4.0, 2.0, 5.0, 5.0, 0.0, 0.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 3.0, 0.0, 5.0, 5.0, 5.0, 5.0, 5.0, 3.0, 5.0, 5.0, 3.0, 5.0, 5.0, 1.0, 5.0, 0.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 0.0, 3.0, 0.0, 3.0, 0.0, 5.0, 0.0, 5.0, 0.0, 4.0, 4.0, 3.0, 5.0, 0.0, 5.0, 0.0, 5.0], [5.0, 3.0, 3.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 3.0, 4.0, 5.0, 0.0, 0.0, 5.0, 0.0, 3.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 4.0, 4.0, 0.0, 4.0, 4.0, 2.0, 4.0, 0.0, 0.0, 3.0, 0.0, 5.0, 5.0, 4.0, 3.0, 5.0, 0.0, 4.0, 3.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.0, 3.0, 4.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0], [3.0, 1.0, 2.0, 2.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 4.0, 2.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 5.0, 1.0, 0.0, 0.0, 0.0, 4.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 4.0, 0.0, 2.0, 2.0, 4.0, 3.0, 0.0, 5.0, 2.0, 0.0, 4.0, 0.0, 5.0, 2.0, 3.0, 4.0, 5.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0, 4.0, 4.0, 4.0, 0.0, 2.0, 5.0, 5.0, 5.0, 4.0, 4.0, 0.0, 0.0], [5.0, 4.0, 5.0, 2.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 4.0, 3.0, 4.0, 5.0, 5.0, 5.0, 0.0, 5.0, 3.0, 2.0, 3.0, 1.0, 5.0, 5.0, 0.0, 4.0, 5.0, 4.0, 4.0, 5.0, 5.0, 3.0, 3.0, 0.0, 2.0, 4.0, 0.0, 4.0, 3.0, 4.0, 0.0, 1.0, 3.0, 3.0, 2.0, 3.0, 4.0, 3.0, 4.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 4.0, 0.0, 4.0, 4.0, 4.0, 0.0, 5.0, 0.0, 5.0, 4.0, 5.0, 4.0, 5.0, 4.0, 2.0, 0.0], [4.0, 3.0, 0.0, 3.0, 5.0, 0.0, 4.0, 3.0, 0.0, 5.0, 5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 0.0, 4.0, 0.0, 5.0, 0.0, 4.0, 0.0, 5.0, 0.0, 4.0, 0.0, 5.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 3.0, 5.0, 0.0, 3.0, 0.0, 4.0, 3.0, 2.0, 0.0, 1.0, 3.0, 0.0, 4.0, 2.0, 1.0, 5.0, 5.0, 5.0, 3.0, 3.0, 5.0, 5.0, 4.0, 4.0, 4.0, 3.0, 5.0, 3.0, 0.0, 4.0, 0.0, 2.0, 0.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0], [5.0, 4.0, 5.0, 2.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 4.0, 3.0, 4.0, 5.0, 5.0, 5.0, 0.0, 5.0, 3.0, 2.0, 3.0, 1.0, 5.0, 5.0, 0.0, 4.0, 5.0, 4.0, 4.0, 5.0, 5.0, 3.0, 3.0, 0.0, 2.0, 4.0, 0.0, 4.0, 3.0, 4.0, 0.0, 1.0, 3.0, 3.0, 2.0, 3.0, 4.0, 3.0, 4.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 4.0, 0.0, 4.0, 4.0, 4.0, 0.0, 5.0, 0.0, 5.0, 4.0, 5.0, 4.0, 5.0, 4.0, 2.0, 0.0], [3.0, 5.0, 5.0, 4.0, 4.0, 0.0, 0.0, 5.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 4.0, 5.0, 5.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 4.0, 5.0, 0.0, 5.0, 5.0, 1.0, 5.0], [5.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 5.0, 3.0, 0.0, 0.0, 4.0, 2.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 5.0, 3.0, 3.0, 0.0, 0.0, 4.0, 3.0, 4.0, 4.0, 0.0, 3.0, 5.0, 0.0, 4.0, 3.0, 3.0, 0.0, 5.0, 0.0, 0.0, 0.0, 5.0, 5.0, 5.0, 0.0, 5.0, 5.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0], [4.0, 4.0, 5.0, 4.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 5.0, 4.0, 0.0, 3.0, 4.0, 0.0, 0.0, 4.0, 4.0, 4.0, 3.0, 0.0, 4.0, 0.0, 0.0, 3.0, 4.0, 3.0, 0.0, 5.0, 0.0, 1.0, 4.0, 0.0, 5.0, 4.0, 3.0, 5.0, 3.0, 3.0, 2.0, 4.0, 3.0, 3.0, 3.0, 5.0, 0.0, 4.0, 0.0, 2.0, 3.0, 4.0, 4.0, 5.0, 0.0, 4.0, 5.0, 5.0, 4.0, 5.0, 5.0, 0.0, 0.0], [4.0, 5.0, 4.0, 4.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 5.0, 4.0, 5.0, 0.0, 5.0, 0.0, 4.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 5.0, 0.0, 5.0, 4.0, 0.0, 0.0], [0.0, 0.0, 5.0, 4.0, 2.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 2.0, 3.0, 4.0, 0.0, 4.0, 4.0, 5.0, 4.0, 5.0, 4.0, 0.0, 0.0], [3.0, 5.0, 3.0, 0.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 0.0, 5.0, 5.0, 5.0, 5.0, 0.0, 3.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 5.0, 5.0, 3.0, 0.0, 5.0, 5.0, 5.0, 0.0, 5.0, 5.0, 3.0, 5.0, 5.0, 0.0, 0.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0], [4.0, 0.0, 4.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 3.0, 0.0, 3.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 4.0, 4.0, 4.0, 5.0, 0.0, 4.0, 4.0, 4.0, 0.0, 0.0], [2.0, 1.0, 0.0, 0.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 2.0, 0.0, 5.0, 0.0, 4.0, 5.0, 2.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0], [5.0, 4.0, 3.0, 4.0, 4.0, 5.0, 5.0, 5.0, 4.0, 0.0, 5.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 5.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 4.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 4.0, 4.0, 2.0, 5.0, 0.0, 0.0, 4.0, 3.0, 3.0, 5.0, 4.0, 0.0, 4.0, 0.0, 4.0, 5.0, 5.0, 4.0, 3.0, 0.0, 4.0, 5.0, 0.0, 0.0, 5.0, 0.0, 4.0, 5.0, 5.0, 5.0, 0.0, 4.0, 0.0, 0.0], [0.0, 0.0, 4.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 4.0, 0.0, 4.0, 5.0, 3.0, 0.0, 2.0, 4.0, 0.0, 0.0], [0.0, 3.0, 4.0, 4.0, 0.0, 0.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 3.0, 0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 5.0, 5.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 4.0, 4.0, 4.0, 0.0, 5.0, 4.0, 4.0, 5.0, 0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 4.0, 0.0, 5.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 3.0, 0.0, 3.0, 4.0, 0.0, 0.0], [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 4.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 2.0, 3.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 1.0, 2.0, 2.0, 1.0, 1.0, 2.0, 3.0, 0.0, 2.0, 0.0, 4.0, 0.0, 0.0, 0.0, 1.0, 4.0, 3.0, 0.0, 1.0, 3.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 4.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 4.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0, 0.0, 0.0, 2.0, 0.0, 2.0, 3.0, 1.0, 0.0, 0.0, 2.0, 1.0, 4.0, 2.0, 0.0, 2.0, 3.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 5.0, 5.0, 0.0, 4.0, 4.0, 4.0, 0.0, 2.0, 5.0, 0.0, 0.0], [4.0, 3.0, 5.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 5.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 3.0, 1.0, 0.0, 3.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 3.0, 4.0, 0.0, 3.0, 2.0, 0.0, 0.0], [3.0, 3.0, 2.0, 2.0, 5.0, 0.0, 3.0, 4.0, 2.0, 0.0, 3.0, 5.0, 3.0, 4.0, 3.0, 3.0, 3.0, 5.0, 0.0, 3.0, 0.0, 3.0, 5.0, 5.0, 3.0, 4.0, 3.0, 0.0, 0.0, 5.0, 3.0, 2.0, 3.0, 0.0, 3.0, 3.0, 2.0, 3.0, 3.0, 3.0, 0.0, 2.0, 3.0, 2.0, 3.0, 0.0, 0.0, 3.0, 4.0, 5.0, 5.0, 5.0, 3.0, 3.0, 5.0, 3.0, 3.0, 4.0, 5.0, 3.0, 0.0, 4.0, 3.0, 0.0, 4.0, 4.0, 0.0, 5.0, 3.0, 4.0, 4.0, 4.0, 1.0, 0.0, 3.0], [0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 2.0, 0.0, 0.0, 0.0, 2.0, 3.0, 4.0, 0.0, 0.0, 2.0, 0.0, 4.0, 0.0, 5.0, 0.0, 2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [4.0, 0.0, 3.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 2.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 3.0, 2.0, 0.0, 3.0, 2.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 2.0, 4.0, 0.0, 0.0, 4.0, 4.0, 2.0, 4.0, 5.0, 0.0, 0.0], [0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 4.0, 0.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 4.0, 4.0, 5.0, 4.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0], [4.0, 4.0, 0.0, 0.0, 5.0, 5.0, 4.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 5.0, 0.0, 3.0, 5.0, 4.0, 4.0, 5.0, 0.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 4.0, 5.0, 5.0, 0.0, 2.0, 0.0, 0.0, 3.0, 4.0, 5.0, 0.0, 0.0, 4.0, 4.0, 0.0, 5.0, 0.0, 4.0, 4.0, 4.0, 0.0, 5.0, 4.0, 4.0, 0.0, 5.0], [2.0, 2.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 4.0, 0.0, 0.0, 4.0, 4.0, 0.0, 4.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 3.0, 4.0, 3.0, 0.0, 4.0, 4.0, 0.0, 0.0], [0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 3.0, 0.0, 3.0, 4.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0], [0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0], [5.0, 5.0, 5.0, 3.0, 5.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 5.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 5.0, 5.0, 5.0, 0.0, 0.0], [1.0, 5.0, 5.0, 2.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 3.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 4.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 0.0, 5.0, 5.0, 5.0, 4.0, 4.0, 2.0, 0.0, 4.0, 0.0, 3.0, 4.0, 0.0, 5.0, 4.0, 4.0, 0.0, 0.0, 3.0, 4.0, 4.0, 5.0, 4.0, 0.0, 5.0, 4.0, 0.0, 0.0], [0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 5.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 2.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 0.0, 0.0], [0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 2.0, 3.0, 0.0, 0.0], [2.0, 2.0, 2.0, 0.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 0.0, 0.0, 4.0, 4.0, 1.0, 5.0, 5.0, 2.0, 5.0, 1.0, 1.0, 5.0, 4.0, 5.0, 4.0, 5.0, 5.0, 1.0, 3.0, 0.0, 3.0, 3.0, 1.0, 3.0, 2.0, 0.0, 1.0, 4.0, 0.0, 1.0, 2.0, 1.0, 3.0, 4.0, 3.0, 0.0, 3.0, 4.0, 5.0, 5.0, 0.0, 3.0, 4.0, 4.0, 4.0, 4.0, 5.0, 2.0, 5.0, 4.0, 4.0, 3.0, 2.0, 4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 5.0, 3.0, 4.0, 5.0], [0.0, 0.0, 0.0, 1.0, 0.0, 4.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 4.0, 3.0, 0.0, 4.0, 4.0, 0.0, 3.0, 3.0, 5.0, 0.0, 3.0, 3.0, 0.0, 5.0], [0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 5.0, 4.0, 0.0, 5.0, 4.0, 4.0, 0.0, 5.0, 4.0, 0.0, 0.0], [5.0, 4.0, 3.0, 3.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 5.0, 0.0, 3.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 5.0, 0.0, 5.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 5.0, 0.0, 3.0, 0.0, 0.0, 5.0, 3.0, 4.0, 5.0, 4.0, 0.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 0.0, 0.0, 3.0, 5.0, 3.0, 5.0, 5.0, 0.0, 3.0, 5.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0], [2.0, 2.0, 2.0, 0.0, 3.0, 5.0, 3.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 2.0, 4.0, 3.0, 0.0, 4.0, 0.0, 0.0, 3.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 4.0, 0.0, 0.0, 0.0, 2.0, 0.0, 2.0, 3.0, 4.0, 0.0, 0.0, 1.0, 0.0, 4.0, 0.0, 5.0, 0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 1.0, 2.0, 0.0, 5.0, 0.0, 3.0, 4.0, 1.0, 2.0, 0.0, 0.0], [3.0, 5.0, 4.0, 3.0, 0.0, 0.0, 5.0, 5.0, 5.0, 4.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 5.0, 0.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 4.0, 0.0, 5.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 2.0, 3.0, 0.0, 4.0, 5.0, 5.0, 4.0, 0.0, 5.0, 4.0, 3.0, 4.0, 3.0, 4.0, 5.0, 3.0, 0.0, 5.0, 0.0, 5.0, 5.0, 3.0, 0.0], [0.0, 4.0, 4.0, 0.0, 4.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 5.0, 4.0, 4.0, 0.0, 0.0, 4.0, 4.0, 3.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 5.0, 5.0, 0.0, 0.0, 3.0, 0.0, 4.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 4.0, 4.0, 0.0, 4.0, 4.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0], [2.0, 0.0, 3.0, 3.0, 0.0, 0.0, 2.0, 0.0, 2.0, 2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 2.0, 2.0, 3.0, 0.0, 0.0, 2.0, 2.0, 1.0, 3.0, 0.0, 0.0, 3.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 4.0, 0.0, 1.0, 3.0, 2.0, 0.0, 1.0, 2.0, 2.0, 1.0, 2.0, 4.0, 0.0, 1.0, 0.0, 0.0, 2.0, 2.0, 4.0, 3.0, 3.0, 3.0, 3.0, 4.0, 0.0, 2.0, 3.0, 3.0, 0.0], [4.0, 4.0, 5.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 5.0, 0.0, 4.0, 4.0, 3.0, 0.0, 2.0, 0.0, 3.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 3.0, 3.0, 0.0, 3.0, 4.0, 5.0, 4.0, 4.0, 4.0, 0.0, 0.0], [3.0, 2.0, 2.0, 1.0, 5.0, 0.0, 5.0, 4.0, 0.0, 5.0, 3.0, 5.0, 4.0, 5.0, 3.0, 3.0, 5.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 0.0, 4.0, 4.0, 0.0, 5.0, 5.0, 2.0, 0.0, 0.0, 2.0, 4.0, 4.0, 1.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 5.0, 0.0, 4.0, 5.0, 5.0, 4.0, 4.0, 2.0, 4.0, 5.0, 0.0, 5.0, 3.0, 5.0, 0.0, 5.0, 3.0, 5.0, 0.0, 5.0, 0.0, 5.0, 5.0, 4.0, 4.0, 4.0, 1.0, 0.0, 4.0], [4.0, 2.0, 1.0, 4.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 0.0, 3.0, 0.0, 3.0, 0.0, 3.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 5.0, 5.0, 0.0, 5.0, 4.0, 3.0, 2.0, 5.0, 3.0, 5.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, 0.0, 2.0, 0.0, 4.0, 5.0, 0.0, 4.0, 4.0, 4.0, 3.0, 5.0, 3.0, 5.0, 3.0, 0.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 3.0, 0.0, 5.0], [4.0, 5.0, 3.0, 3.0, 4.0, 3.0, 5.0, 5.0, 0.0, 5.0, 4.0, 0.0, 4.0, 3.0, 4.0, 3.0, 3.0, 4.0, 5.0, 3.0, 4.0, 0.0, 5.0, 4.0, 3.0, 0.0, 3.0, 0.0, 0.0, 5.0, 5.0, 4.0, 3.0, 5.0, 4.0, 3.0, 0.0, 0.0, 4.0, 3.0, 5.0, 3.0, 3.0, 3.0, 4.0, 4.0, 0.0, 3.0, 3.0, 5.0, 5.0, 4.0, 4.0, 4.0, 5.0, 3.0, 5.0, 5.0, 5.0, 4.0, 0.0, 5.0, 5.0, 2.0, 3.0, 3.0, 3.0, 5.0, 4.0, 5.0, 4.0, 4.0, 3.0, 0.0, 5.0], [0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 3.0, 4.0, 5.0, 0.0, 0.0, 5.0, 5.0, 3.0, 5.0, 5.0, 0.0, 0.0], [4.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 5.0, 0.0, 0.0, 0.0, 1.0, 0.0, 3.0, 2.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [3.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 3.0, 4.0, 0.0, 5.0, 5.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 2.0, 0.0, 0.0, 0.0, 4.0, 4.0, 3.0, 4.0, 0.0, 0.0, 0.0, 4.0, 5.0, 4.0, 3.0, 2.0, 4.0, 0.0, 4.0, 0.0, 3.0, 0.0, 4.0, 0.0, 0.0, 0.0, 2.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 3.0, 0.0, 0.0], [0.0, 0.0, 3.0, 0.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 4.0, 2.0, 0.0, 0.0, 0.0, 4.0, 0.0, 5.0, 2.0, 5.0, 5.0, 5.0, 2.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 0.0, 3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 4.0, 5.0, 5.0, 0.0, 1.0, 5.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 5.0, 3.0, 0.0, 5.0, 5.0, 0.0, 4.0, 5.0, 0.0, 4.0, 5.0, 3.0, 0.0, 5.0], [0.0, 0.0, 2.0, 1.0, 0.0, 0.0, 5.0, 2.0, 3.0, 3.0, 2.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 2.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 1.0, 0.0, 0.0, 2.0, 4.0, 4.0, 0.0, 0.0, 5.0, 2.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.0, 5.0, 5.0, 2.0, 3.0, 0.0, 5.0], [0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 4.0, 0.0, 4.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0], [5.0, 4.0, 5.0, 0.0, 5.0, 0.0, 5.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 5.0, 5.0, 5.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 1.0, 5.0, 0.0, 0.0, 0.0, 5.0, 5.0, 5.0, 5.0, 0.0, 4.0, 5.0, 5.0, 0.0, 5.0, 0.0, 5.0, 0.0, 0.0, 5.0, 5.0, 5.0, 5.0, 0.0, 5.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0], [5.0, 4.0, 4.0, 3.0, 2.0, 3.0, 3.0, 4.0, 4.0, 2.0, 2.0, 2.0, 1.0, 5.0, 4.0, 5.0, 0.0, 2.0, 0.0, 5.0, 2.0, 3.0, 2.0, 5.0, 1.0, 2.0, 4.0, 0.0, 1.0, 5.0, 5.0, 4.0, 4.0, 1.0, 4.0, 2.0, 2.0, 3.0, 5.0, 1.0, 0.0, 2.0, 0.0, 2.0, 1.0, 0.0, 0.0, 5.0, 1.0, 5.0, 5.0, 4.0, 2.0, 3.0, 4.0, 5.0, 3.0, 5.0, 2.0, 5.0, 3.0, 3.0, 4.0, 3.0, 5.0, 5.0, 2.0, 4.0, 4.0, 4.0, 1.0, 3.0, 4.0, 0.0, 2.0], [0.0, 5.0, 5.0, 4.0, 2.0, 0.0, 5.0, 0.0, 3.0, 2.0, 3.0, 0.0, 1.0, 4.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 5.0, 0.0, 1.0, 3.0, 3.0, 0.0, 0.0, 5.0, 4.0, 2.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 3.0, 1.0, 5.0, 5.0, 0.0, 2.0, 3.0, 0.0, 4.0, 1.0, 5.0, 0.0, 3.0, 0.0, 3.0, 5.0, 5.0, 5.0, 3.0, 4.0, 5.0, 5.0, 5.0, 0.0, 5.0, 5.0, 0.0, 0.0], [4.0, 3.0, 3.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 2.0, 3.0, 0.0, 0.0, 4.0, 0.0, 5.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 5.0, 5.0, 4.0, 3.0, 0.0, 0.0, 3.0, 0.0, 4.0, 4.0, 2.0, 0.0, 1.0, 1.0, 3.0, 5.0, 3.0, 0.0, 5.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 5.0, 4.0, 1.0, 5.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 4.0, 4.0, 0.0, 0.0, 5.0, 0.0, 1.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 5.0, 5.0, 4.0, 0.0, 1.0, 1.0, 5.0, 0.0, 5.0, 0.0, 5.0, 5.0, 0.0, 5.0, 0.0, 5.0, 5.0, 5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 4.0, 0.0, 5.0], [4.0, 4.0, 3.0, 4.0, 5.0, 4.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 3.0, 5.0, 4.0, 5.0, 5.0, 5.0, 4.0, 3.0, 5.0, 4.0, 4.0, 4.0, 5.0, 5.0, 4.0, 3.0, 4.0, 5.0, 5.0, 4.0, 4.0, 5.0, 2.0, 4.0, 5.0, 3.0, 2.0, 5.0, 4.0, 4.0, 3.0, 3.0, 3.0, 4.0, 5.0, 3.0, 5.0, 5.0, 5.0, 3.0, 5.0, 2.0, 4.0, 5.0, 4.0, 5.0, 5.0, 5.0, 4.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0], [3.0, 4.0, 3.0, 0.0, 3.0, 5.0, 5.0, 2.0, 3.0, 0.0, 2.0, 3.0, 4.0, 3.0, 3.0, 2.0, 5.0, 0.0, 0.0, 5.0, 0.0, 4.0, 2.0, 3.0, 0.0, 2.0, 0.0, 1.0, 5.0, 5.0, 3.0, 1.0, 0.0, 0.0, 3.0, 2.0, 0.0, 3.0, 3.0, 2.0, 0.0, 4.0, 0.0, 2.0, 4.0, 0.0, 3.0, 3.0, 2.0, 5.0, 5.0, 3.0, 3.0, 2.0, 2.0, 3.0, 3.0, 2.0, 1.0, 1.0, 0.0, 3.0, 4.0, 1.0, 0.0, 5.0, 0.0, 1.0, 1.0, 4.0, 5.0, 3.0, 0.0, 0.0, 0.0], [0.0, 4.0, 4.0, 0.0, 4.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 5.0, 4.0, 4.0, 0.0, 0.0, 4.0, 4.0, 3.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 5.0, 5.0, 0.0, 0.0, 3.0, 0.0, 4.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 4.0, 4.0, 0.0, 4.0, 4.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0], [3.0, 4.0, 2.0, 4.0, 4.0, 1.0, 5.0, 5.0, 5.0, 0.0, 4.0, 5.0, 5.0, 5.0, 4.0, 5.0, 2.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 3.0, 1.0, 0.0, 4.0, 4.0, 4.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 1.0, 5.0, 5.0, 3.0, 0.0, 3.0, 2.0, 4.0, 4.0, 3.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 2.0, 3.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 2.0, 5.0, 5.0, 3.0, 5.0, 5.0, 0.0, 5.0, 5.0, 5.0, 5.0, 5.0, 2.0, 0.0, 4.0], [4.0, 3.0, 3.0, 1.0, 3.0, 0.0, 5.0, 0.0, 5.0, 0.0, 4.0, 0.0, 4.0, 4.0, 4.0, 5.0, 5.0, 2.0, 2.0, 5.0, 0.0, 5.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 4.0, 0.0, 4.0, 5.0, 4.0, 5.0, 5.0, 2.0, 0.0, 3.0, 4.0, 5.0, 4.0, 5.0, 3.0, 4.0, 4.0, 5.0, 4.0, 4.0, 3.0, 4.0, 0.0, 4.0, 0.0, 2.0, 2.0, 5.0, 5.0, 4.0, 3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0], [0.0, 0.0, 4.0, 4.0, 5.0, 0.0, 4.0, 4.0, 0.0, 5.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 5.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 5.0, 0.0, 4.0, 5.0, 5.0, 0.0, 0.0, 0.0, 4.0, 5.0, 0.0, 5.0, 0.0, 0.0, 0.0, 5.0, 0.0, 3.0, 5.0, 5.0, 0.0, 4.0, 4.0, 5.0, 4.0, 4.0, 3.0, 0.0, 0.0], [0.0, 5.0, 2.0, 3.0, 5.0, 4.0, 0.0, 0.0, 0.0, 5.0, 2.0, 4.0, 0.0, 4.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 3.0, 4.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 5.0, 5.0, 4.0, 0.0, 2.0, 0.0, 5.0, 0.0, 5.0, 0.0, 1.0, 0.0, 5.0, 0.0, 2.0, 0.0, 5.0, 0.0, 5.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 5.0], [0.0, 4.0, 3.0, 0.0, 5.0, 0.0, 5.0, 4.0, 0.0, 0.0, 5.0, 5.0, 0.0, 5.0, 5.0, 5.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 3.0, 5.0, 0.0, 5.0, 5.0, 0.0, 5.0, 5.0, 0.0, 5.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 3.0, 5.0, 5.0, 5.0, 0.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 5.0, 5.0, 3.0, 5.0, 5.0, 3.0, 5.0, 0.0, 5.0], [0.0, 3.0, 1.0, 0.0, 4.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 3.0, 0.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 5.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 4.0, 2.0, 0.0, 0.0, 3.0, 3.0, 0.0, 4.0, 5.0, 0.0, 0.0], [5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 3.0, 0.0, 4.0, 5.0, 0.0, 5.0, 5.0, 0.0, 0.0, 3.0, 4.0, 5.0, 5.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 5.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 4.0, 5.0, 5.0, 5.0, 3.0, 4.0, 0.0, 0.0], [3.0, 4.0, 5.0, 3.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 5.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 4.0, 3.0, 3.0, 0.0, 5.0, 0.0, 4.0, 0.0, 4.0, 0.0, 3.0, 5.0, 3.0, 3.0, 0.0, 4.0, 3.0, 0.0, 5.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 2.0, 4.0, 0.0, 5.0, 3.0, 5.0, 4.0, 4.0, 3.0, 3.0, 3.0, 5.0, 5.0, 5.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 3.0, 3.0, 5.0, 4.0, 2.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0], [4.0, 5.0, 3.0, 3.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 3.0, 0.0, 0.0, 4.0, 5.0, 5.0, 0.0, 5.0, 0.0, 3.0, 0.0, 4.0, 4.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 5.0, 3.0, 5.0, 5.0, 3.0, 5.0, 0.0, 3.0, 4.0, 5.0, 2.0, 2.0, 4.0, 3.0, 0.0, 0.0, 4.0, 3.0, 5.0, 5.0, 5.0, 4.0, 3.0, 4.0, 0.0, 5.0, 3.0, 4.0, 5.0, 3.0, 0.0, 4.0, 3.0, 5.0, 5.0, 3.0, 5.0, 5.0, 4.0, 4.0, 5.0, 0.0, 5.0, 0.0, 5.0], [3.0, 4.0, 4.0, 2.0, 5.0, 4.0, 5.0, 5.0, 0.0, 5.0, 4.0, 4.0, 3.0, 4.0, 5.0, 5.0, 0.0, 4.0, 5.0, 4.0, 5.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 0.0, 5.0, 4.0, 3.0, 5.0, 4.0, 0.0, 4.0, 4.0, 3.0, 3.0, 0.0, 2.0, 3.0, 4.0, 3.0, 3.0, 0.0, 5.0, 2.0, 4.0, 5.0, 5.0, 4.0, 4.0, 4.0, 5.0, 4.0, 0.0, 3.0, 5.0, 3.0, 5.0, 3.0, 5.0, 3.0, 4.0, 4.0, 5.0, 5.0, 4.0, 5.0, 5.0, 4.0, 4.0, 0.0, 5.0], [3.0, 3.0, 5.0, 0.0, 5.0, 5.0, 5.0, 2.0, 0.0, 2.0, 5.0, 0.0, 5.0, 5.0, 1.0, 5.0, 0.0, 2.0, 0.0, 2.0, 1.0, 2.0, 5.0, 5.0, 0.0, 5.0, 5.0, 0.0, 5.0, 5.0, 5.0, 3.0, 0.0, 4.0, 3.0, 3.0, 0.0, 5.0, 2.0, 5.0, 4.0, 0.0, 0.0, 0.0, 1.0, 3.0, 5.0, 0.0, 4.0, 5.0, 3.0, 5.0, 3.0, 4.0, 0.0, 5.0, 1.0, 5.0, 1.0, 2.0, 5.0, 0.0, 2.0, 4.0, 3.0, 5.0, 5.0, 4.0, 5.0, 5.0, 4.0, 5.0, 4.0, 0.0, 0.0], [0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 5.0, 4.0, 0.0, 5.0, 4.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0], [5.0, 5.0, 5.0, 5.0, 5.0, 0.0, 5.0, 5.0, 0.0, 5.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 0.0, 5.0, 0.0, 5.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 5.0, 5.0, 5.0, 0.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 0.0, 5.0, 0.0, 5.0, 5.0, 0.0, 5.0, 5.0, 5.0, 5.0, 5.0, 0.0, 5.0, 0.0, 5.0], [4.0, 4.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 4.0, 0.0, 0.0, 3.0, 3.0, 4.0, 0.0, 0.0, 3.0, 0.0, 4.0, 5.0, 4.0, 0.0, 3.0, 0.0, 4.0, 3.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 2.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [4.0, 4.0, 2.0, 2.0, 5.0, 0.0, 5.0, 5.0, 0.0, 4.0, 4.0, 5.0, 4.0, 4.0, 0.0, 5.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 4.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 4.0, 4.0, 0.0, 5.0, 5.0, 5.0, 3.0, 2.0, 5.0, 4.0, 5.0, 4.0, 5.0, 4.0, 0.0, 0.0, 4.0, 4.0, 3.0, 3.0, 0.0, 5.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0], [3.0, 4.0, 0.0, 3.0, 5.0, 5.0, 5.0, 4.0, 0.0, 0.0, 2.0, 0.0, 3.0, 5.0, 3.0, 5.0, 0.0, 0.0, 0.0, 4.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 2.0, 3.0, 5.0, 0.0, 3.0, 4.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 4.0, 5.0, 4.0, 2.0, 0.0, 3.0, 4.0, 3.0, 0.0, 5.0, 0.0, 3.0, 0.0, 3.0, 0.0, 4.0, 4.0, 4.0, 3.0, 4.0, 5.0, 3.0, 0.0, 5.0, 4.0, 0.0, 0.0], [0.0, 4.0, 3.0, 3.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 5.0], [3.0, 4.0, 2.0, 3.0, 4.0, 5.0, 4.0, 5.0, 5.0, 0.0, 4.0, 3.0, 2.0, 4.0, 5.0, 3.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 5.0, 4.0, 3.0, 3.0, 4.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 2.0, 4.0, 3.0, 2.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 2.0, 3.0, 4.0, 5.0, 4.0, 3.0, 3.0, 3.0, 4.0, 4.0, 5.0, 5.0, 3.0, 0.0, 4.0, 4.0, 4.0, 3.0, 4.0, 0.0, 4.0, 5.0, 3.0, 3.0, 5.0, 4.0, 4.0, 4.0], [2.0, 3.0, 4.0, 1.0, 4.0, 0.0, 5.0, 2.0, 0.0, 3.0, 4.0, 5.0, 3.0, 5.0, 3.0, 3.0, 0.0, 3.0, 0.0, 5.0, 0.0, 0.0, 3.0, 5.0, 4.0, 4.0, 3.0, 0.0, 0.0, 5.0, 5.0, 0.0, 3.0, 0.0, 2.0, 2.0, 0.0, 3.0, 2.0, 3.0, 0.0, 2.0, 4.0, 3.0, 3.0, 0.0, 0.0, 2.0, 1.0, 5.0, 5.0, 2.0, 0.0, 4.0, 3.0, 5.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 3.0, 3.0, 5.0, 5.0, 5.0, 0.0, 4.0, 3.0, 0.0, 3.0], [3.0, 3.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 2.0, 0.0, 0.0, 0.0, 2.0, 5.0, 3.0, 0.0, 0.0, 2.0, 0.0, 4.0, 0.0, 3.0, 1.0, 3.0, 0.0, 0.0, 0.0, 3.0, 4.0, 0.0, 0.0, 2.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0], [0.0, 3.0, 2.0, 4.0, 4.0, 0.0, 4.0, 0.0, 5.0, 0.0, 4.0, 0.0, 5.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0, 5.0, 5.0, 4.0, 4.0, 0.0, 0.0, 5.0, 5.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 4.0, 3.0, 5.0, 4.0, 5.0, 5.0, 3.0, 4.0, 5.0, 5.0, 4.0, 5.0, 3.0, 4.0, 0.0, 5.0, 4.0, 3.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 4.0, 3.0, 0.0, 4.0], [3.0, 4.0, 5.0, 3.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 5.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 4.0, 3.0, 3.0, 0.0, 5.0, 0.0, 4.0, 0.0, 4.0, 0.0, 3.0, 5.0, 3.0, 3.0, 0.0, 4.0, 3.0, 0.0, 5.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 2.0, 4.0, 0.0, 5.0, 3.0, 5.0, 4.0, 4.0, 3.0, 3.0, 3.0, 5.0, 5.0, 5.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 3.0, 3.0, 5.0, 4.0, 2.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0], [4.0, 2.0, 1.0, 1.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 5.0, 4.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0], [2.0, 3.0, 5.0, 5.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.0, 5.0, 0.0, 3.0, 0.0, 3.0, 3.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 1.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 3.0, 4.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0, 3.0, 5.0, 0.0, 1.0, 5.0, 0.0, 0.0], [5.0, 5.0, 1.0, 2.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 4.0, 0.0, 5.0, 5.0, 5.0, 4.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 3.0, 4.0, 5.0, 5.0, 0.0, 3.0, 5.0, 0.0, 0.0, 5.0, 4.0, 3.0, 2.0, 0.0, 2.0, 3.0, 5.0, 4.0, 5.0, 2.0, 5.0, 5.0, 5.0, 2.0, 3.0, 5.0, 5.0, 5.0, 4.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 3.0, 5.0, 0.0, 5.0], [5.0, 2.0, 5.0, 3.0, 3.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 5.0, 0.0, 3.0, 0.0, 4.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 3.0, 3.0, 5.0, 3.0, 0.0, 3.0, 0.0, 4.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 3.0, 2.0, 5.0, 4.0, 0.0, 5.0, 4.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0], [5.0, 4.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 2.0, 3.0, 5.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 4.0, 4.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0], [3.0, 4.0, 4.0, 2.0, 5.0, 4.0, 5.0, 5.0, 0.0, 5.0, 4.0, 4.0, 3.0, 4.0, 5.0, 5.0, 0.0, 4.0, 5.0, 4.0, 5.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 0.0, 5.0, 4.0, 3.0, 5.0, 4.0, 0.0, 4.0, 4.0, 3.0, 3.0, 0.0, 2.0, 3.0, 4.0, 3.0, 3.0, 0.0, 5.0, 2.0, 4.0, 5.0, 5.0, 4.0, 4.0, 4.0, 5.0, 4.0, 0.0, 3.0, 5.0, 3.0, 5.0, 3.0, 5.0, 3.0, 4.0, 4.0, 5.0, 5.0, 4.0, 5.0, 5.0, 4.0, 4.0, 0.0, 5.0], [2.0, 1.0, 3.0, 1.0, 3.0, 0.0, 5.0, 5.0, 4.0, 5.0, 0.0, 0.0, 5.0, 4.0, 4.0, 2.0, 0.0, 0.0, 5.0, 5.0, 4.0, 2.0, 5.0, 3.0, 5.0, 0.0, 4.0, 4.0, 1.0, 4.0, 4.0, 3.0, 0.0, 5.0, 3.0, 4.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 0.0, 4.0, 2.0, 4.0, 4.0, 3.0, 3.0, 4.0, 5.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 0.0, 5.0, 4.0, 0.0, 5.0, 5.0, 3.0, 4.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, 3.0, 0.0, 5.0], [0.0, 0.0, 4.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 3.0, 5.0, 5.0, 4.0, 4.0, 4.0, 5.0, 5.0, 4.0, 0.0, 0.0, 0.0], [4.0, 2.0, 1.0, 1.0, 3.0, 3.0, 5.0, 2.0, 2.0, 5.0, 2.0, 1.0, 1.0, 2.0, 3.0, 2.0, 1.0, 3.0, 5.0, 4.0, 5.0, 2.0, 4.0, 2.0, 5.0, 4.0, 5.0, 3.0, 1.0, 4.0, 3.0, 2.0, 5.0, 5.0, 2.0, 2.0, 4.0, 1.0, 2.0, 5.0, 1.0, 5.0, 4.0, 2.0, 2.0, 3.0, 5.0, 2.0, 4.0, 5.0, 4.0, 3.0, 5.0, 4.0, 4.0, 3.0, 1.0, 2.0, 3.0, 2.0, 3.0, 5.0, 4.0, 5.0, 5.0, 5.0, 2.0, 3.0, 5.0, 5.0, 5.0, 4.0, 5.0, 0.0, 5.0], [5.0, 4.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 3.0, 0.0, 0.0, 4.0, 0.0, 3.0, 4.0, 5.0, 0.0, 3.0, 0.0, 0.0, 0.0, 5.0, 5.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 3.0, 0.0, 5.0, 4.0, 5.0, 0.0, 5.0, 4.0, 0.0, 5.0, 0.0, 5.0, 0.0, 3.0, 4.0, 5.0, 4.0, 5.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 4.0, 0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0], [0.0, 5.0, 5.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 5.0, 5.0, 3.0, 0.0, 4.0, 0.0, 5.0, 0.0, 4.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 3.0, 0.0, 4.0, 0.0, 0.0], [3.0, 4.0, 5.0, 3.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 5.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 4.0, 3.0, 3.0, 0.0, 5.0, 0.0, 4.0, 0.0, 4.0, 0.0, 3.0, 5.0, 3.0, 3.0, 0.0, 4.0, 3.0, 0.0, 5.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 2.0, 4.0, 0.0, 5.0, 3.0, 5.0, 4.0, 4.0, 3.0, 3.0, 3.0, 5.0, 5.0, 5.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 3.0, 3.0, 5.0, 4.0, 2.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0], [3.0, 3.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0, 5.0, 3.0, 0.0, 3.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 2.0, 4.0, 5.0, 5.0, 0.0, 4.0, 3.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 5.0, 3.0, 0.0, 4.0, 0.0, 5.0, 0.0, 5.0, 3.0, 5.0, 5.0, 0.0, 0.0], [5.0, 4.0, 1.0, 4.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 5.0, 0.0, 5.0, 5.0, 4.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 3.0, 0.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 4.0, 3.0, 3.0, 0.0, 2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 2.0, 2.0, 5.0, 5.0, 0.0, 3.0, 3.0, 0.0, 2.0, 3.0, 3.0, 0.0, 0.0], [0.0, 5.0, 4.0, 2.0, 5.0, 5.0, 4.0, 5.0, 0.0, 0.0, 5.0, 4.0, 0.0, 5.0, 5.0, 5.0, 0.0, 5.0, 0.0, 0.0, 0.0, 5.0, 0.0, 5.0, 4.0, 4.0, 0.0, 5.0, 0.0, 4.0, 5.0, 5.0, 0.0, 3.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 4.0, 4.0, 5.0, 5.0, 5.0, 1.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 3.0, 4.0, 5.0, 5.0, 5.0, 5.0, 0.0, 5.0, 4.0, 2.0, 4.0, 0.0, 3.0, 5.0, 0.0, 5.0], [0.0, 3.0, 0.0, 0.0, 5.0, 0.0, 4.0, 3.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 4.0, 4.0, 4.0, 0.0, 2.0, 0.0, 5.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 3.0, 0.0, 3.0, 4.0, 3.0, 3.0, 0.0, 3.0], [3.0, 3.0, 3.0, 2.0, 5.0, 2.0, 5.0, 0.0, 0.0, 4.0, 0.0, 0.0, 2.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 5.0, 5.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 3.0, 0.0, 4.0, 0.0, 0.0, 3.0, 0.0, 0.0, 5.0, 3.0, 1.0, 5.0, 4.0, 0.0, 0.0, 3.0, 2.0, 5.0, 0.0, 3.0, 0.0, 3.0, 5.0, 0.0, 0.0, 0.0, 5.0, 3.0, 0.0, 4.0, 5.0, 5.0, 0.0, 4.0, 5.0, 0.0, 0.0], [3.0, 4.0, 4.0, 3.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 5.0, 4.0, 3.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 2.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 3.0, 5.0, 0.0, 4.0, 4.0, 3.0, 3.0, 0.0, 4.0, 3.0, 4.0, 4.0, 5.0, 0.0, 3.0, 0.0, 3.0, 4.0, 0.0, 4.0, 3.0, 0.0, 4.0, 5.0, 4.0, 0.0, 5.0, 3.0, 0.0, 0.0]]
gen_scores_last = generate_scores(last_data)
song_index = ['You Was Right - Lil Uzi Vert', 'Moonlight - XXXTENTACION', 'Closer - The Chainsmokers', 'Lonely - Justin Bieber, Benny Blanco', 'Heavenly Father - Isaiah Rashad', 'Hell N Back - Bakar', 'Cigarette Daydreams - Cage The Elephant', 'Peach - Kevin Abstract', 'She Wants My Money - Dominic Fike', 'Peach Pit - Peach Pit', 'Love is Only a Feeling - Joey Bada$$', 'Shadow Man - Noname, Smino, etc', 'HONEST - Baby Keem', 'Forbidden Fruit - J Cole, Kendrick Lamar', 'Trippy - Anderson Paak, J Cole', 'Righteous Minds - Joey Bada$$', 'Suburban, Pt. 2 - 22Gz', 'Ace - Noname, Smino, Saba', "Cool with You - Her's", 'Location - Playboi Carti', 'I Think I Like When It Rains - Willis', 'Through Da Storm - Polo G', 'Free - Louis the Child, Drew Love', "Wat's Wrong - Isaiah Rashad, Kendrick Lamar, Zacari", 'Float On - Modest Mouse', 'Sandstorm - Mereba, JID', 'Inside Friend - Leon Bridges, John Mayer', 'N Side - Steve Lacy', 'Big Dreams - Bakar', 'American Boy - Kanye West, Estelle', 'Ni**as in Paris - JAY-Z, Kanye West', 'Woah - Rich the Kid, Miguel, Ty Dolla', 'Stuck In the Middle - Tai Verdes', 'All That and More (Sailboat) - Rainbow Kitten Surprise', 'Handgun - DaBaby, NoCap, Polo G', 'Heebiejeebies - Amine, Kehlani', 'To Me - Chet Faker', 'Ned Flanders - MadeinTYO, A$AP Ferg, Stwo', 'Feeling - Juice WRLD', 'Ladies, Ladies, Ladies - Dreamville, JID, T.I.', 'Walked In - Bankroll Fresh, Travis Porter, Street Money Boochie', 'Took Her To The O - King Von', 'Vossi Bop - Stormzy', 'Make No Sense - YoungBoy Never Broke Again', 'Vibin In This Bih - Kodak Black, Gucci Mane', 'Same Team - Labrinth, Stefflon Don', '1901 - Phoenix', 'Walk Em Down - NLE Choppa, Roddy Ricch', 'I Might Need Security - Chance the Rapper', 'Pursuit Of Happiness - Kid Cudi', 'Nights - Frank Ocean', 'Proud Of U - EARTHGANG, Young Thug', 'Tomorrow - Sean Kingston', 'Black Beatles - Rae Sremmurd', 'The Bird - Anderson .Paak', 'Excuse Me -A$AP Rocky', 'Slide - H.E.R., YG', 'Slide - Calvin Harris, Frank Ocean, Migos', 'Japanese Denim - Daniel Caesar', 'Redemption - Drake', "Vibin' Out - French Kiwi Juice", 'Corduroy Dreams - Rex Orange County', 'Feel It Still - Portugal. The Man', 'Sparks - Coldplay', 'Valerie - Amy Winehouse', "Don't Let me Down - The Beatles", 'Foreplay - Jalen Santoy', 'Paper Planes - M.I.A.', 'Sunday Morning - Maroon 5', 'Kids - MGMT', 'Last Nite - The Strokes', 'Island In The Sun - Weezer', 'Use Somebody - Kings of Leon', 'Life Inside - Verzache', 'Lucy - Still Woozy, ODIE']


In [73]:
recommendations(last_data, gen_scores_last, 132)

[('Hell N Back - Bakar', 3.944344039187249),
 ('Japanese Denim - Daniel Caesar', 3.7154966483790837),
 ('She Wants My Money - Dominic Fike', 3.6768119646751316),
 ('Righteous Minds - Joey Bada$$', 3.6685367302884755),
 ('Slide - H.E.R., YG', 3.6582355495271583)]